<a href="https://colab.research.google.com/github/Remdeht/ia_detector/blob/master/IA_Classification_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the necessary libraries.

In [2]:
!git clone https://github.com/Remdeht/ia_detector.git ia

Cloning into 'ia'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 64 (delta 25), reused 57 (delta 18), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [0]:
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 7 (delta 5), reused 7 (delta 5), pack-reused 0
Unpacking objects: 100% (7/7), done.
From https://github.com/Remdeht/ia_detector
   fd72eae..62a1604  master     -> origin/master
Updating fd72eae..62a1604
Fast-forward
 gee_functions/classification.py | 174 +++++++++++++++++++++------------
 gee_functions/export.py         |  50 ++++++----
 gee_functions/validation.py     |  16 +++-
 gee_functions/visualization.py  | 208 ++++++++++++++++++++--------------------
 4 files changed, 260 insertions(+), 188 deletions(-)


In [3]:
 !pip install monthdelta

  Created wheel for monthdelta: filename=MonthDelta-0.9.1-cp36-none-any.whl size=3111 sha256=b645673f878510d8f9e4585d907052daa31f0896cb8c7fe607a33f1a71892dc4
  Stored in directory: /root/.cache/pip/wheels/5e/e9/68/c0cb0cefe6fdd49ce42a99196b272748d2d823c18c7af612cf
Successfully built monthdelta


In [0]:
import ee
import itertools
import sys
import numpy as np
from ia.gee_functions.classification import create_features, create_training_areas, classify_irrigated_areas, \
    join_seasonal_irrigated_areas
from ia.gee_functions.constants import GEE_USER_PATH
from ia.gee_functions.export import track_task
from ia.gee_functions import visualization
from datetime import datetime

Initialize the GEE, if never used before authentication needs be done first using ee.Authenticate().

In [0]:
try:
  ee.Initialize()
except FileNotFoundError:
  ee.Authenticate()
  ee.Initialize()

In [0]:
GEE_USER_PATH = 'users/remdeht'

Specify the area of interest name, this name will be used during the naming of the intermediary and final result.



In [0]:
areas = {
    'Campo de Cartagena':'cdc',
    'Murcia Region':'rdm',
    'Segura Watershed':'cds',
    'Custom Rectangular Extent':'',
    'Other GEE Vector':''
}

In [0]:
area = "Campo de Cartagena" #@param ["Campo de Cartagena", "Murcia Region", "Segura Watershed", "Custom Rectangular Extent", "Other GEE Vector"]

In [0]:
area_of_interest = areas[area]

In [0]:
if not area_of_interest == '':
  aoi = ee.FeatureCollection(f'users/Postm087/vector/outline/outline_{area_of_interest}')
else:
  aoi_name = 'custom_polygon' #@param {type:"string"}
  area_of_interest = 'csp' #@param {type:"string"}
  lat_min = 37.656830  #@param {type:"number"}
  lat_max = 37.784704  #@param {type:"number"}
  lng_min = -1.046193 #@param {type:"number"}
  lng_max = -0.832859 #@param {type:"number"}
  aoi = ee.FeatureCollection(ee.Feature(ee.Geometry.Polygon([
                   [lng_min, lat_max],
                   [lng_max, lat_max],
                   [lng_max, lat_min],
                   [lng_min, lat_min],
                   [lng_min, lat_max]
  ])))

aoi_coordinates = aoi.geometry().bounds().getInfo()['coordinates']
aoi_centroid = aoi.geometry().bounds().centroid(1).coordinates().getInfo()

In [14]:
layer = {
    'Area of Interest':aoi.getMapId({'color':'White'})
}

map = visualization.create_folium_map(layer, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map

Select the timeperiod for which the classification will be performed. Time period must be 1 or 2 years.

In [0]:
start_date = "2019-01-01" #@param {type:"date"}
end_date = "2020-01-01" #@param {type:"date"}

start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
year = str(int(round(np.mean([start_date_dt.year, end_date_dt.year]))))[-2:]

In [0]:
classification_period = {year:(start_date, end_date)}

Next up, the outline for the area of interest is selected from a GEE featurecollection. 

Statistical feature maps maps to be generated for classification

In [0]:
stats = ['median', 'min', 'max']
stats_combos = list(itertools.combinations(stats, 3))

The folder locations, for the storage and loading of data

In [0]:
crop_data_folder = f'{GEE_USER_PATH}/raster/crop_data/{area_of_interest}/'
training_data_folder = f'{GEE_USER_PATH}/raster/training_areas/{area_of_interest}/'
results_folder = f'{GEE_USER_PATH}/raster/results/'

Next, the feature data for classification is generated.  

In [18]:
for year in classification_period:
  for season in ['winter', 'summer']:
    export_feature_task = create_features(
        classification_period[year],
        aoi,
        aoi_name=area_of_interest,
        year_string=year,
        season=season,
        user_path=GEE_USER_PATH
        ) 

Export started for crop_data/cdc/crop_data_winter_min_median_max_cdc_20
Export started for crop_data/cdc/crop_data_summer_min_median_max_cdc_20


In [0]:
track_task(export_feature_task)  # tracks the status of the export task

Running Task (0 min)
Running Task (1 min)
Running Task (2 min)
Running Task (3 min)
Running Task (4 min)
Running Task (5 min)
Running Task (6 min)
Running Task (7 min)
Running Task (8 min)


The feature data maps are generated, let's perform a quick check and load the RGB composites into a folium map. 

In [0]:

summer = ee.Image(f"{crop_data_folder}crop_data_summer_min_median_max_{area_of_interest}_{year}")
winter = ee.Image(f"{crop_data_folder}crop_data_winter_min_median_max_{area_of_interest}_{year}")
images = {
      'Summer Feature Data RGB': summer.getMapId(visualization.vis_params_rgb_ls457(bands=['red', 'green', 'blue'])), 
      'Winter Feature Data RGB': winter.getMapId(visualization.vis_params_rgb_ls457(bands=['red', 'green', 'blue'])) 
}
map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map

Now that the feature data is generated and exported to the GEE training areas can be selected via thresholding. 

In [0]:
for year in classification_period:
  for season in ['winter', 'summer']:
    export_training_areas_task = create_training_areas(
          aoi,
          ee.Image(f"{crop_data_folder}crop_data_{season}_min_median_max_{area_of_interest}_{year}"),
          aoi_name=area_of_interest,
          year_string=year,
          season=season,
          user_path=GEE_USER_PATH
          )

Export task started for year: csp 19.
Export task started for year: csp 19.


In [0]:
track_task(export_training_areas_task)  # tracks the export task

 Running Task (0 min)
 Running Task (1 min)
 Task Completed, runtime: 2


True

Let's show the training areas in a folium map, to check the result. 

In [0]:
training_classes = {
    0:'Unused',
    1:'Forest',
    2:'Scrubs',
    3:'Rainfed Crops and Trees',
    4:'Greenhouses',
    5:'Irrigated Herbaceous Crops',
    6:'Irrigated Trees',
    7: 'Water Bodies',
}

summer_training = ee.Image(f'{training_data_folder}training_areas_summer_{area_of_interest}_{year}')
winter_training = ee.Image(f'{training_data_folder}training_areas_winter_{area_of_interest}_{year}')
images_training = {
      'Summer Feature Training Areas': summer_training.getMapId(visualization.vis_rf_classification(band='training')),
      'Winter Feature Training Areas': winter_training.getMapId(visualization.vis_rf_classification(band='training')) 
  }
map = visualization.create_folium_map(images_training, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
visualization.create_categorical_legend(map, visualization.vis_rf_classification()['palette'][:7] + ['4837BC'], training_classes)
map

Now that the feature data and training area maps are complete the random forest classification can be done.  

In [0]:
no_trees = 350  # Number of trees for the RF classifier to run
vps = 2  # Variables per split
bf = .6  # bagging factor

for year in classification_period:
  for season in ['winter', 'summer']:
    for combo in stats_combos:
      # load the feature data maps      
        crop_data_image = ee.Image(f"{crop_data_folder}crop_data_{season}_min_median_max_{area_of_interest}_{year}")

        bands_to_select = ['red', 'green', 'blue', 'thermal', 'swir','.*std.*', 'TWI']
        stat_bands = [f'.*{s}.*' for s in list(combo)]
        bands_to_select += stat_bands # bands to select for classification

        crop_data_image = crop_data_image.select(bands_to_select)
        classification_name = "_".join(combo)

        classification_name += f'_{season}'
        
        training_image = ee.Image(
            f'{GEE_USER_PATH}/raster/training_areas/{area_of_interest}/training_areas_{season}_{area_of_interest}_{year}')

        classification_task = classify_irrigated_areas(
            crop_data_image,
            training_image,
            aoi,
            classification_name,
            aoi_name=area_of_interest,
            year=year,
            user_path=GEE_USER_PATH,
            clf='random_forest',
            no_trees=no_trees,
            bag_fraction=bf,
            vps=vps
            )

Export started. AOI: csp.
Year: 19.
Classification algorithm: random_forest.
Features used median_min_max_winter.

Export started. AOI: csp.
Year: 19.
Classification algorithm: random_forest.
Features used median_min_max_summer.



In [0]:
track_task(classification_task)

 Running Task (0 min)
 Running Task (1 min)
 Running Task (2 min)
 Running Task (3 min)
 Task Completed, runtime: 4


True

Now that the classification has been completed, the results can be inspected again using a folium map. 

In [0]:

summer_clf = ee.Image(f'{results_folder}random_forest/{area_of_interest}/ia_random_forest_median_min_max_summer_{no_trees}tr_{vps}vps_{int(bf*100)}bf_{area_of_interest}_{year}')
winter_clf = ee.Image(f'{results_folder}random_forest/{area_of_interest}/ia_random_forest_median_min_max_winter_{no_trees}tr_{vps}vps_{int(bf*100)}bf_{area_of_interest}_{year}')
images_clf = {
    'Summer RF Classification results': summer_clf.getMapId(visualization.vis_rf_classification()),
    'Winter RF Classification results': winter_clf.getMapId(visualization.vis_rf_classification())
}

visualization.create_folium_map(images_clf, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%') 

Finally, after the classification is finished the results are combined using an or operation to create the final classifcation map.

In [0]:
ia_summer = summer_clf.select('irrigated_area')
ia_winter = winter_clf.select('irrigated_area')
task = join_seasonal_irrigated_areas(
    ia_summer,
    ia_winter,
    area_of_interest,
    year,
    aoi_coordinates,
    export_method='asset',
    user_path=GEE_USER_PATH

  )

In [0]:
track_task(task)

 Running Task (0 min)
 Task Completed, runtime: 1


True

Let's show the final result!

In [0]:
ia_classes = {
        0: 'Not Irrigated',
        1: 'Year Round Irrigated Trees',
        2: 'Year Round Irrigated Crops',
        3: 'Summer Irrigated Trees',
        4: 'Summer Irrigated Crops',
        5: 'Winter Irrigated Trees',
        6: 'Winter Irrigated Crops',
        7: 'Uncertain Areas',
    }

ia_year = ee.Image(f'{results_folder}irrigated_area/{area_of_interest}/irrigated_areas_{area_of_interest}_{year}').clip(aoi)
images_results = {
      'Irrigated Areas Overview': ia_year.getMapId(visualization.vis_irrigated_area_map())
  }
map = visualization.create_folium_map(images_results, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
visualization.create_categorical_legend(map, visualization.vis_irrigated_area_map()['palette'], ia_classes)
